In [11]:
import pandas as pd
from database.adatabase import ADatabase
from processor.processor import Processor
from sklearn.cluster import KMeans

In [7]:
market = ADatabase("market")
russell1000 = pd.read_html("https://en.wikipedia.org/wiki/Russell_1000_Index")[2].rename(columns={"Symbol":"ticker"})

In [8]:
market.connect()
overview = []
for ticker in russell1000["ticker"]:
    prices = Processor.column_date_processing(market.query("prices",{"ticker":ticker})).sort_values("date")
    overview.append({
        "ticker":ticker,"risk":prices["adjclose"].rolling(5).std().mean(),"return":prices["adjclose"].pct_change(5).mean()    })
market.disconnect()

In [9]:
grouping_data = pd.DataFrame(overview)

In [14]:
grouping_data
X = grouping_data[["risk","return"]]
kmeans = KMeans(n_clusters=10, random_state=42)
grouping_data["cluster"] = kmeans.fit_predict(X)

In [17]:
grouping_data

,ticker,risk,return,cluster
0,TXG,1.244707,-0.000041,0
1,MMM,1.524087,0.003322,6
2,AOS,0.978468,0.005523,0
3,AAON,1.276156,0.010593,6
4,ABT,1.144463,0.002193,0
...,...,...,...,...
1004,ZION,0.981599,0.001490,0
1005,ZTS,2.486494,0.003272,6
1006,ZM,1.376173,0.000025,6
1007,ZI,0.651160,-0.010320,0


In [16]:
db = ADatabase("sapling")
db.connect()
db.store("clusters",grouping_data)
db.disconnect()